In [2]:
import glob
from operator import itemgetter
import json

metas =  glob.glob('../tmp/*meta*.json')
def get_meta(meta_name):
    with open(meta_name, 'r') as f:
        return json.load(f)
    
def summarize(meta):
    bill = meta['bill']
    state, bill_id, status_date, legiscan_bill_id = itemgetter('state', 'bill_number', 'status_date', 'bill_id')(bill)
    
    return {
        'state': state,
        'bill_id': bill_id,
        'status_date': status_date,
        'legiscan_bill_id': legiscan_bill_id,
    }

with open('../artifacts/aggregate.json', 'w') as f:
    json.dump([summarize(get_meta(meta)) for meta in metas], f, indent=2)